In [88]:
# Initial and final results
GNN_LSTM = 2.2851
initial_xgb = 2.0279
initial_rf = 1.9293
best_xgb = 1.8649
final_result = 1.8086

# Calculate percentage improvement
GNN_LSTM_improvement = ((GNN_LSTM - final_result) / GNN_LSTM) * 100
initial_xgb_improvement = ((initial_xgb - final_result) / initial_xgb) * 100
initial_rf_improvement = ((initial_rf - final_result) / initial_rf) * 100
best_xgb_improvement = ((best_xgb - final_result) / best_xgb) * 100

print(GNN_LSTM_improvement)
print(initial_xgb_improvement)
print(initial_rf_improvement)
print(best_xgb_improvement)


GNN_LSTM = 4.7387
initial_xgb = 2.9
initial_rf = 2.7385
best_xgb = 2.6611
final_result = 2.5861

# Calculate percentage improvement
GNN_LSTM_improvement = ((GNN_LSTM - final_result) / GNN_LSTM) * 100
initial_xgb_improvement = ((initial_xgb - final_result) / initial_xgb) * 100
initial_rf_improvement = ((initial_rf - final_result) / initial_rf) * 100
best_xgb_improvement = ((best_xgb - final_result) / best_xgb) * 100
print()
print(GNN_LSTM_improvement)
print(initial_xgb_improvement)
print(initial_rf_improvement)
print(best_xgb_improvement)

GNN_LSTM = 2.2851
initial_xgb = 2.0279
weather_lag = 1.9185
best_xgb = 1.8649
final_result = 1.8086

# Calculate percentage improvement
GNN_LSTM_improvement = ((GNN_LSTM - initial_xgb) / GNN_LSTM) * 100
initial_xgb_improvement = ((initial_xgb - weather_lag) / initial_xgb) * 100
weather_improvement = ((weather_lag - best_xgb) / weather_lag) * 100
best_xgb_improvement = ((best_xgb - final_result) / best_xgb) * 100
print()
print(GNN_LSTM_improvement)
print(initial_xgb_improvement)
print(weather_improvement)
print(best_xgb_improvement)

final_improvement = 20.852479103759133
print()
print(GNN_LSTM_improvement/final_improvement)
print(initial_xgb_improvement/final_improvement)
print(weather_improvement/final_improvement)
print(best_xgb_improvement/final_improvement)

20.852479103759133
10.81414270920656
6.256155082154151
3.018928628880906

45.425960706522886
10.824137931034478
5.565090377944133
2.8183833752959204

11.255524922322879
5.394743330538968
2.793849361480328
3.018928628880906

0.5397691500524662
0.2587099262248604
0.13398164062788454
0.1447755259151261


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
!unzip /content/drive/MyDrive/Thesis/Data/compressed_data.zip
!unzip /content/drive/MyDrive/Thesis/Data/cosine_similarity.zip

Archive:  /content/drive/MyDrive/Thesis/Data/compressed_data.zip
  inflating: demand_graphs.pkl.npz   
  inflating: final_model_input_partial_scale_4.csv  
Archive:  /content/drive/MyDrive/Thesis/Data/cosine_similarity.zip
  inflating: cosine_similarity.csv   


In [7]:
!unzip /content/drive/MyDrive/Thesis/Data/returns.zip

Archive:  /content/drive/MyDrive/Thesis/Data/returns.zip
  inflating: demand_model_o_xgb_01_11.sav  
  inflating: test_predictions_o_xgb_01_11.csv  


In [8]:
import math
import torch
import pickle
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import joblib
import gc
from sklearn.decomposition import PCA
# check xgboost version
from xgboost import XGBRegressor


#torch stuff
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

#from torch_geometric.utils import dense_to_sparse


import lightgbm as lgb
from lightgbm import LGBMRegressor
import holidays

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

directory = '/content/drive/MyDrive/Thesis'
data_dir = directory + "/Data"
models_dir = directory + "/models"

In [9]:
from hyperopt.pyll.base import scope
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [10]:

execfile('/content/drive/MyDrive/Thesis/models_training/model_training.py')

Using device: cpu


In [11]:
target_scaler = joblib.load(f'{models_dir}/target_scaler.sav')

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [12]:

#################################
## Cosine Similarity
#################################

cosine_similarity = pd.read_csv(f'cosine_similarity.csv').drop(columns=['Unnamed: 0'])
cols = [f"cosine_sim_{i}" for i in range(183)]
cosine_similarity[cols] = cosine_similarity[cols].astype('float32')
cosine_similarity['started_at_hourly'] = pd.to_datetime(cosine_similarity['started_at_hourly'])

#################################
## PCA
#################################

PCA_DIM = 1

pca_test = cosine_similarity[cosine_similarity['started_at_hourly'] >= pd.to_datetime("2024-01-01 00:00:00")]
pca_train = cosine_similarity[cosine_similarity['started_at_hourly'] < pd.to_datetime("2024-01-01 00:00:00")]

del cosine_similarity

pca_train_time_clusters = pca_train[['started_at_hourly', 'start_station_cluster']]
pca_test_time_clusters = pca_test[['started_at_hourly', 'start_station_cluster']]

pca_train.drop(columns=['started_at_hourly', 'start_station_cluster'], inplace=True)
pca_test.drop(columns=['started_at_hourly', 'start_station_cluster'], inplace=True)

pca = PCA(n_components=PCA_DIM, svd_solver='arpack')

m = pca.fit_transform(pca_train[cols])
n = pca.transform(pca_test[cols])
del pca_train

n.shape


df_train_pca = pd.DataFrame(m, columns=[f"cosine_sim_pca_{i}" for i in range(PCA_DIM)])
df_test_pca = pd.DataFrame(n, columns=[f"cosine_sim_pca_{i}" for i in range(PCA_DIM)])

del m
del n

df_train_pca[['started_at_hourly', 'start_station_cluster']] = pca_train_time_clusters[['started_at_hourly', 'start_station_cluster']]
df_test_pca[['started_at_hourly', 'start_station_cluster']] = pca_test_time_clusters[['started_at_hourly', 'start_station_cluster']]

#################################
#################################

df = pd.read_csv(f'final_model_input_partial_scale_4.csv')

suf = "_normal_xgb_01_11"

returns = pd.read_csv(f"{models_dir}/test_returns_predictions{suf}.csv").rename(
    columns={
        "end_station_cluster": "start_station_cluster",
        "pred": "returns"
        }
    )[['started_at_hourly', 'returns', 'start_station_cluster']]

#################################
#################################

df = df.merge(returns, on=['started_at_hourly', 'start_station_cluster'], how='left').fillna(0)

df['started_at_hourly'] = pd.to_datetime(df['started_at_hourly'])
df = df.sort_values(by=['start_station_cluster', 'started_at_hourly'])

df_test = df[df['started_at_hourly'] >= pd.to_datetime("2024-01-01 00:00:00")]
df_train = df[df['started_at_hourly'] < pd.to_datetime("2024-01-01 00:00:00")]

del df
del returns

df_train = df_train.merge(df_train_pca, on=[
    'started_at_hourly', 'start_station_cluster'
    ], how='left').fillna(0)

df_test = df_test.merge(df_test_pca, on=[
    'started_at_hourly', 'start_station_cluster'
    ], how='left').fillna(0)


df_hold_out = df_test[df_test['started_at_hourly'] >= pd.to_datetime("2024-03-25 00:00:00")]
df_test = df_test[df_test['started_at_hourly'] < pd.to_datetime("2024-03-25 00:00:00")]

#del df_train_pca
#del df_test_pca


#################################
#################################

In [13]:
df_hold_out.shape

(21780, 148)

In [14]:
df_test.shape

(260500, 148)

In [15]:
del df_train_pca
del df_test_pca
del pca_test
del pca_test_time_clusters
del pca_train_time_clusters

# GNN Variance Embedding

In [16]:
FEATURES = ['start_station_cluster',# 'started_at_year',
            'started_at_month', 'started_at_day',
            'started_at_hour',
            'started_at_week',
            'started_at_quarter',
            'started_at_dayofweek',
            'is_holiday',
            #'flag_added',
            "temp",
            "dwpt",
            "rhum",
            "prcp",
            "wdir",
            "wspd",
            "pres",
            "coco",
            "demand_lag_1_h",
            "demand_lag_2_h",
            "demand_lag_24_h",
            "temp_lag_1_h",
            "temp_lag_2_h",
            "temp_lag_24_h",
            "prcp_lag_1_h",
            "prcp_lag_2_h",
            "prcp_lag_24_h",
            "rhum_lag_1_h",
            "rhum_lag_2_h",
            "rhum_lag_24_h",
            "wspd_lag_1_h",
            "wspd_lag_2_h",
            "wspd_lag_24_h",
            # "coco_lag_1_h",
            # "coco_lag_2_h",
            # "coco_lag_24_h",
            "mean_gnn_cluster_demand_1h",
            "total_gnn_cluster_demand_1h",
            "total_demand_1h",
            "demand_degrees_1h",
            "returns"
]
#FEATURES.extend([f"dim_mean_{i}" for i in range(50)])
FEATURES.extend([f"cosine_sim_pca_{i}" for i in range(PCA_DIM)])

In [39]:
def train_model(df_train, df_test, FEATURES, model='xgb', epochs=50, batch_size=32, tuning=False):
    TARGET = 'demand'
    TARGET_TRAIN = 'demand_target'
    X_train = df_train[FEATURES]
    y_train = df_train[TARGET_TRAIN]

    X_test = df_test[FEATURES]
    y_test = df_test[TARGET]

    overall_zero_scores = pd.DataFrame()
    non_zero_scores = pd.DataFrame()
    zero_scores = pd.DataFrame()
    preds_list = []
    #10, 20, 30, 42, 50, 60, 70, 80, 90,
    for state in [0]:
        if model=='rf':
            reg = RandomForestRegressor(random_state=state)
            reg.fit(X_train, y_train)

        if model=='xgb':
            if tuning:
               reg = tuning_xgb(X_train, y_train, X_test, y_test)
            else:
               if state==0:
                   state = None
               reg = XGBRegressor(random_state=state)
               reg.fit(X_train, y_train)

        if model=='lgbm':
            # Define the parameter grid
            if tuning:
                param_grid = {
                    'learning_rate': [0.01, 0.3, 0.05, 0.1, 0.2],
                    'num_leaves': [10, 20, 31, 40, 50, 60],
                    'max_depth': [-1, 10, 20, 30, 50],
                    'feature_fraction': [0.6, 0.8, 1.0],
                    'bagging_fraction': [0.6, 0.8, 1.0],
                    'n_estimators': [20, 40, 60, 80, 100, 120, 140]
                }

                # Initialize the LightGBM model
                model = lgb.LGBMRegressor(objective='regression', metric='rmse')

                # Perform grid search
                grid_search = RandomizedSearchCV(
                    estimator=model,
                    param_distributions=param_grid,
                    scoring='neg_mean_squared_error',
                    cv=3,
                    verbose=1,
                    n_iter=10,
                )
                grid_search.fit(X_train, y_train)

                # Best parameters
                #print("Best parameters:", grid_search.best_params_)
                reg = LGBMRegressor(**grid_search.best_params_)
                reg.fit(X_train, y_train)
            else:
                reg = LGBMRegressor(random_state=state)
                reg.fit(X_train, y_train)


        if model=='lstm':

            dataset = TimeSeriesDataset(X_train.to_numpy(), y_train.to_numpy())
            test_dataset = TimeSeriesDataset(X_test.to_numpy(), y_test.to_numpy())

            # Create DataLoader
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
            reg = fit_lstm(dataloader, FEATURES, batch_size, epochs)

            output_df = test_lstm(reg, test_dataloader, FEATURES, batch_size)

            return reg, output_df

        y_pred = target_scaler.inverse_transform([reg.predict(X_test)])
        preds_out = X_test.copy()
        preds_out['actual_demand'] = y_test
        preds_out['pred'] = y_pred[0]

        if model != 'rf':
            preds_list.append(preds_out)

        non_zero = preds_out.query('actual_demand != 0')
        zeros = preds_out.query('actual_demand == 0')

        mse_score = mean_squared_error(y_test, y_pred[0])
        rmse_score = np.sqrt(mse_score)
        mae_score = mean_absolute_error(y_test, y_pred[0])
        mape_score = mean_absolute_percentage_error(y_test+1, y_pred[0]+1)
        overall_zero_scores = pd.concat([overall_zero_scores, pd.DataFrame({'mse': [mse_score], 'rmse': [rmse_score], 'mae': [mae_score], 'mape': [mape_score]})])

        mse_score = mean_squared_error(non_zero['actual_demand'], non_zero['pred'])
        rmse_score = np.sqrt(mse_score)
        mae_score = mean_absolute_error(non_zero['actual_demand'], non_zero['pred'])
        mape_score = mean_absolute_percentage_error(non_zero['actual_demand']+1, non_zero['pred']+1)
        non_zero_scores = pd.concat([non_zero_scores, pd.DataFrame({'mse': [mse_score], 'rmse': [rmse_score], 'mae': [mae_score], 'mape': [mape_score]})])


        mse_score = mean_squared_error(zeros['actual_demand'], zeros['pred'])
        rmse_score = np.sqrt(mse_score)
        mae_score = mean_absolute_error(zeros['actual_demand'], zeros['pred'])
        mape_score = mean_absolute_percentage_error(zeros['actual_demand']+1, zeros['pred']+1)
        zero_scores = pd.concat([zero_scores, pd.DataFrame({'mse': [mse_score], 'rmse': [rmse_score], 'mae': [mae_score], 'mape': [mape_score]})])

    overall_out = {
        "mse": overall_zero_scores['mse'].mean(),
        "rmse": overall_zero_scores['rmse'].mean(),
        "mae": overall_zero_scores['mae'].mean(),
        "mape": overall_zero_scores['mape'].mean()
    }

    non_zero_out = {
        "mse": non_zero_scores['mse'].mean(),
        "rmse": non_zero_scores['rmse'].mean(),
        "mae": non_zero_scores['mae'].mean(),
        "mape": non_zero_scores['mape'].mean()
    }


    zero_out = {
        "mse": zero_scores['mse'].mean(),
        "rmse": zero_scores['rmse'].mean(),
        "mae": zero_scores['mae'].mean(),
        "mape": zero_scores['mape'].mean()
    }

    return reg, preds_out, preds_list, overall_out, non_zero_out, zero_out

#### Slope Based

In [40]:
exclude_features_slope = {

    "4.39 - inf": {
        "clusters": [35, 65, 69, 79, 94],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp",
            "prcp_lag_1_h", "wspd", "wspd_lag_1_h", "temp_lag_2_h",
            "wspd_lag_24_h", "wspd_lag_2_h", "started_at_month",
            "dwpt", "cosine_sim_pca_0", "rhum", "temp_lag_24_h"
        ],

        "most_optimal": [ #28.79711292470574
          'is_holiday', 'prcp', 'wspd', 'wspd_lag_1_h', 'temp_lag_2_h',
          'wspd_lag_24_h', 'started_at_month', 'dwpt', 'cosine_sim_pca_0',
          'temp_lag_24_h'
        ]
    },

    "3.85 - 4.39": {
        "clusters": [71, 86, 91, 105, 122],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday",  "wspd",
            "rhum", "wspd_lag_2_h", "prcp", "temp_lag_2_h", "dwpt", "temp",
            "started_at_month", "wspd_lag_1_h", "temp_lag_24_h"
        ],

        "most_optimal": [ #16.086499775426713
            'started_at_quarter', 'prcp_lag_2_h', 'wspd', 'temp',
            'started_at_month', 'wspd_lag_1_h', 'temp_lag_24_h'
        ]
    },

    "3.21 - 3.85": {
        "clusters": [53, 62, 99, 145, 154],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday",  "wspd",
            "rhum", "wspd_lag_2_h", "temp", "started_at_month", "prcp",
            "temp_lag_2_h", "temp_lag_1_h", "cosine_sim_pca_0"
        ],

        "most_optimal": [ #9.284845055288182
              'started_at_quarter', 'prcp_lag_2_h', 'rhum', 'wspd_lag_2_h',
              'started_at_month', 'prcp', 'temp_lag_2_h', 'cosine_sim_pca_0'
        ]
    },

    "2.84 - 3.21": {
        "clusters": [34, 42, 57, 59, 60, 72, 81, 88, 89, 102],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday",  "wspd",
            "temp", "rhum", "started_at_month", "wspd_lag_2_h", "temp_lag_2_h",
            "prcp", "wspd_lag_1_h", "temp_lag_1_h", "dwpt", "wdir"
        ],

        "most_optimal": [ #7.0278315540821845
            'started_at_quarter', 'wspd', 'temp', 'rhum', 'started_at_month',
            'wspd_lag_2_h', 'temp_lag_2_h', 'temp_lag_1_h', 'dwpt', 'wdir'
        ]
    },

    "2.38 - 2.84": {
        "clusters": [39, 77, 95, 97, 113, 118, 163],
        "features": [
            "started_at_quarter", "is_holiday", "prcp_lag_2_h", "wspd",
            "rhum", "temp", "started_at_month", "wspd_lag_2_h", "wspd_lag_1_h",
            "temp_lag_2_h", "prcp", "dwpt", "temp_lag_1_h", "rhum_lag_24_h"
        ],

        "most_optimal": [ #5.198946122014834
          'started_at_quarter', 'is_holiday', 'prcp_lag_2_h', 'wspd',
          'started_at_month', 'temp_lag_2_h', 'prcp', 'dwpt', 'rhum_lag_24_h'
        ]
    },

    "2.06 - 2.38": {
        "clusters": [64, 84, 92, 98, 106, 123, 130, 144, 155],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "wspd", "started_at_month",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h", "temp_lag_1_h",
            "wspd_lag_1_h", "prcp", "dwpt", "temp_lag_2_h", "rhum_lag_24_h"
        ],

        "most_optimal": [ #3.5547193565154456
          'started_at_quarter', 'prcp_lag_2_h', 'temp', 'wspd_lag_2_h',
          'temp_lag_1_h', 'dwpt', 'rhum_lag_24_h'
        ]
    },

    "0 - 2.06": {
        "clusters": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                     17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
                     32, 33, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51,
                     52, 54, 55, 56, 58, 61, 63, 66, 67, 68, 70, 73, 74, 75, 76,
                     78, 80, 82, 83, 85, 87, 90, 93, 96, 100, 101, 103, 104,
                     107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 120, 121,
                     124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136,
                     137, 138, 139, 140, 141, 142, 143, 146, 147, 148, 149, 150,
                     151, 152, 153, 156, 157, 158, 159, 160, 161, 162, 164],
        "features": [
            "started_at_quarter", "started_at_month", "wspd", "prcp_lag_2_h",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h",  "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "temp_lag_2_h", "prcp",
            "cosine_sim_pca_0"
        ],

        "most_optimal": [#0.8803045158511106
            'started_at_month', 'rhum', 'temp', 'wspd_lag_2_h',
            'dwpt', 'wspd_lag_1_h', 'temp_lag_2_h', 'prcp', 'cosine_sim_pca_0'
        ]
    },


}

In [41]:
# Overall
# MSE:  3.288441355314878
# RMSE:  1.8134060095066626
# MAE:  0.9587181847445567
# MAPE:  0.37653170668575936

exclude_features_slope = {

    "4.39 - inf": {
        "clusters": [35, 65, 69, 79, 94],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp",
            "prcp_lag_1_h", "wspd", "wspd_lag_1_h", "temp_lag_2_h",
            "wspd_lag_24_h", "wspd_lag_2_h", "started_at_month",
            "dwpt", "cosine_sim_pca_0", "rhum", "temp_lag_24_h"
        ],

        "most_optimal": [ #28.79711292470574
          'is_holiday', 'prcp', 'wspd', 'wspd_lag_1_h', 'temp_lag_2_h',
          'wspd_lag_24_h', 'started_at_month', 'dwpt', 'cosine_sim_pca_0',
          'temp_lag_24_h'
        ]
    },

    "3.85 - 4.39": {
        "clusters": [71, 86, 91, 105, 122],
        "features": [#16.133195292665977
            "started_at_quarter", "prcp_lag_2_h", "is_holiday",  "wspd",
            "rhum", "wspd_lag_2_h", "prcp", "temp_lag_2_h", "dwpt", "temp",
            "started_at_month", "wspd_lag_1_h", "temp_lag_24_h"
        ],

        "most_optimal": [
            'started_at_quarter', 'wspd', 'rhum', 'wspd_lag_2_h', 'temp',
            'started_at_month'
            ]
    },

    "3.21 - 3.85": {
        "clusters": [53, 62, 99, 145, 154],
        "features": [
          "started_at_quarter", "prcp_lag_2_h", "is_holiday",  "wspd",
          "rhum", "wspd_lag_2_h", "temp", "started_at_month", "prcp",  "wspd_lag_1_h",
          "dwpt", "temp_lag_2_h", "temp_lag_1_h"
        ],

        "most_optimal": [ #9.2036380650924
            'started_at_quarter', 'wspd', 'rhum', 'temp', 'started_at_month',
            'prcp', 'wspd_lag_1_h', 'dwpt', 'temp_lag_1_h'
        ]
    },

    "2.3 - 3.21": {
        "clusters": [34, 39, 42, 57, 59, 60, 72, 77, 81, 88, 89, 95, 97, 102,
                     106, 113, 118, 163],
        "features": [
          "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
          "temp", "started_at_month", "wspd_lag_2_h", "temp_lag_2_h",
          "wspd_lag_1_h", "prcp", "temp_lag_1_h", "dwpt", "wdir"
        ],

        "most_optimal": [ #6.113803980329564
            'started_at_quarter', 'prcp_lag_2_h', 'is_holiday', 'temp',
            'wspd_lag_2_h', 'temp_lag_1_h', 'dwpt'
        ]
    },

    "0 - 2.3": {
        "clusters": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                     17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
                     32, 33, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51,
                     52, 54, 55, 56, 58, 61, 63, 64, 66, 67, 68, 70, 73, 74, 75,
                     76, 78, 80, 82, 83, 84, 85, 87, 90, 92, 93, 96, 98, 100,
                     101, 103, 104, 107, 108, 109, 110, 111, 112, 114, 115, 116,
                     117, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131,
                     132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
                     144, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157,
                     158, 159, 160, 161, 162, 164],
        "features": [
            "started_at_quarter", "started_at_month", "wspd", "prcp_lag_2_h",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h",  "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "temp_lag_2_h", "prcp",
            "cosine_sim_pca_0"
        ],

        "most_optimal": [ #1.0583844883909233
            'started_at_month', 'prcp_lag_2_h', 'is_holiday', 'dwpt',
            'temp_lag_1_h', 'temp_lag_2_h', 'cosine_sim_pca_0'
        ]
    },


}

In [42]:
# Overall
# MSE:  3.227538656133565
# RMSE:  1.7965351808783385
# MAE:  0.9535124740154409
# MAPE:  0.3760509598145607

exclude_features_slope = {

    "3.85 - inf": {
        "clusters": [35, 65, 69, 71, 79, 86, 91, 94, 105, 122],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp", "wspd",
            "wspd_lag_1_h", "wspd_lag_2_h", "temp_lag_2_h", "started_at_month",
            "rhum", "dwpt", "wspd_lag_1_h", "temp", "temp_lag_24_h",
            "cosine_sim_pca_0"
        ],
        #"model": "rf",
        "most_optimal": [ #21.627476640892645
            'started_at_quarter', 'prcp_lag_2_h', 'prcp', 'wspd_lag_2_h', 'temp', 'cosine_sim_pca_0'
        ]
    },

    "2.3 - 3.85": {
        "clusters": [34, 39, 42, 53, 57, 59, 60, 62, 72, 77, 81, 88, 89, 95, 97,
                     99, 102, 106, 113, 118, 145, 154, 163],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd",
            "rhum", "temp", "started_at_month", "wspd_lag_2_h",
            "prcp", "wspd_lag_1_h", "temp_lag_2_h", "temp_lag_1_h", "dwpt",
            "wdir"
        ],

        #"model": "rf",
        "most_optimal": [ #6.739131558843824
            'started_at_quarter', 'prcp_lag_2_h', 'wspd', 'rhum',
            'started_at_month', 'wspd_lag_2_h', 'prcp', 'wspd_lag_1_h',
            'temp_lag_2_h', 'temp_lag_1_h'
        ]
    },

    "0 - 2.3": {
        "clusters": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                     17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
                     32, 33, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51,
                     52, 54, 55, 56, 58, 61, 63, 64, 66, 67, 68, 70, 73, 74, 75,
                     76, 78, 80, 82, 83, 84, 85, 87, 90, 92, 93, 96, 98, 100,
                     101, 103, 104, 107, 108, 109, 110, 111, 112, 114, 115, 116,
                     117, 120, 121, 123, 124, 125, 126, 127, 128, 129, 130, 131,
                     132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
                     144, 146, 147, 148, 149, 150, 151, 152, 153, 155, 156, 157,
                     158, 159, 160, 161, 162, 164],
        "features": [
            "started_at_quarter", "started_at_month", "wspd", "prcp_lag_2_h",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h",  "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "temp_lag_2_h", "prcp",
            "cosine_sim_pca_0"
        ],
        #"model": "xgb",

        "most_optimal": [ #1.0583844883909233
            'started_at_month', 'prcp_lag_2_h', 'is_holiday', 'dwpt',
            'temp_lag_1_h', 'temp_lag_2_h', 'cosine_sim_pca_0'
        ]
    },


}

#### 2.0

In [43]:
exclude_features_2 = {
    "4.0 - inf": {
        "clusters": [35, 65, 69, 71, 79, 86, 91, 94],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp",
            "prcp_lag_1_h", "wspd", "wspd_lag_1_h", "temp_lag_2_h",
            "wspd_lag_2_h", "started_at_month", "wspd_lag_24_h", "dwpt",
            "rhum"
        ],

        "most_optimal": [
            'prcp_lag_2_h', 'prcp', 'wspd_lag_2_h', 'wspd_lag_24_h', 'dwpt',
            'rhum'
         ]
    },
    "2.0 - 4.0": {
        "clusters": [34, 39, 42, 53, 57, 59, 60, 62, 64, 72, 77, 81, 84, 88, 89,
                     92, 95, 97, 98, 99, 102, 105, 106, 113, 118, 122, 123, 130,
                     144, 145, 154, 155, 163],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
            "temp", "started_at_month", "wspd_lag_2_h", "prcp", "wspd_lag_1_h",
            "temp_lag_2_h", "temp_lag_1_h", "dwpt", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'started_at_quarter', 'wspd', 'temp', 'started_at_month',
            'wspd_lag_2_h', 'prcp', 'wspd_lag_1_h', 'temp_lag_2_h',
            'temp_lag_1_h', 'cosine_sim_pca_0'
        ]
    },

    "0 - 2.0": {
        "clusters": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                     17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,
                     32, 33, 36, 37, 38, 40, 41, 43, 44, 45, 46, 47, 48, 50, 51,
                     52, 54, 55, 56, 58, 61, 63, 66, 67, 68, 70, 73, 74, 75, 76,
                     78, 80, 82, 83, 85, 87, 90, 93, 96, 100, 101, 103, 104, 107,
                     108, 109, 110, 111, 112, 114, 115, 116, 117, 120, 121, 124,
                     125, 126, 127, 128, 129, 131, 132, 133, 134, 135, 136, 137,
                     138, 139, 140, 141, 142, 143, 146, 147, 148, 149, 150, 151,
                     152, 153, 156, 157, 158, 159, 160, 161, 162, 164],
        "features": [
            "started_at_quarter", "started_at_month", "wspd", "prcp_lag_2_h",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h",  "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "temp_lag_2_h", "prcp",
            "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'prcp_lag_2_h', 'is_holiday', 'rhum', 'temp', 'wspd_lag_2_h',
            'temp_lag_1_h', 'wspd_lag_1_h', 'prcp', 'cosine_sim_pca_0'
        ]
    },

}

#### 1.5


In [44]:
exclude_features_1_5 = {
    "4.5 - inf": {
        "clusters": [35, 65, 79, 94],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp",
            "prcp_lag_1_h", "wspd_lag_1_h", "wspd", "wspd_lag_24_h",
            "temp_lag_2_h", "wspd_lag_2_h", "started_at_month",
            "cosine_sim_pca_0"
        ],

        "most_optimal": [
          'started_at_quarter', 'is_holiday', 'prcp', 'wspd',
          'wspd_lag_24_h', 'temp_lag_2_h'
        ]
    },

    "3.0 - 4.5": {
        "clusters": [42, 53, 62, 69, 71, 81, 86, 88, 91, 99,
                     102, 105, 122, 145, 154],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
            "wspd_lag_2_h", "prcp", "started_at_month", "temp", "temp_lag_2_h",
            "wspd_lag_1_h", "dwpt"
        ],

        "most_optimal": [
            'wspd', 'prcp', 'started_at_month', 'temp_lag_2_h', 'wspd_lag_1_h'
        ]
    },

    "1.5 - 3.0": {
        "clusters": [3, 19, 25, 34, 39, 43, 46, 51, 57, 59, 60, 61, 64, 72, 73,
                     75, 76, 77, 82, 84, 89, 90, 92, 95, 97, 98, 106, 108, 112,
                     113, 114, 115, 116, 118, 120, 123, 129, 130, 136, 140, 141,
                     144, 148, 149, 152, 155, 160, 163, 164],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "wspd", "is_holiday",
            "started_at_month", "rhum", "temp", "wspd_lag_2_h", "temp_lag_1_h",
            "wspd_lag_1_h", "temp_lag_2_h"
        ],

        "most_optimal": [
            'prcp_lag_2_h', 'wspd', 'rhum', 'wspd_lag_2_h'
        ]
    },

    "0 - 1.5": {
        "clusters": [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                     18, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 36,
                     37, 38, 40, 41, 44, 45, 47, 48, 50, 52, 54, 55, 56, 58, 63,
                     66, 67, 68, 70, 74, 78, 80, 83, 85, 87, 93, 96, 100, 101,
                     103, 104, 107, 109, 110, 111, 117, 121, 124, 125, 126, 127,
                     128, 131, 132, 133, 134, 135, 137, 138, 139, 142, 143, 146,
                     147, 150, 151, 153, 156, 157, 158, 159, 161, 162],
        "features": [
            "started_at_quarter", "wspd", "started_at_month", "prcp_lag_2_h",
            "rhum", "is_holiday", "wspd_lag_2_h", "temp", "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "prcp", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'rhum', 'dwpt', 'temp_lag_1_h', 'wspd_lag_1_h', 'cosine_sim_pca_0'
        ]
    },

}







#### 0.75

In [45]:
exclude_features_0_75 = {

    "4.5 - inf": {
        "clusters": [35, 65, 79, 94],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp",
            "prcp_lag_1_h", "wspd_lag_1_h", "wspd", "wspd_lag_24_h",
            "temp_lag_2_h", "wspd_lag_2_h", "started_at_month"
        ],

        "most_optimal": [
          'started_at_quarter', 'is_holiday', 'prcp', 'wspd', 'wspd_lag_24_h',
          'temp_lag_2_h'
        ]
    },

    "3.75 - 4.5": {
        "clusters": [69, 71, 86, 91, 105, 122],
        "features": [ #16.652377277620758
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "prcp",
            "wspd_lag_2_h", "rhum", "temp_lag_2_h", "started_at_month", "dwpt",
            "wspd_lag_1_h", "temp", "temp_lag_24_h", "prcp_lag_1_h"

        ],

        "most_optimal": [
            'started_at_quarter', 'prcp_lag_2_h', 'rhum', 'temp_lag_2_h',
            'started_at_month'
        ]
    },

    "3.0 - 3.75": {
        "clusters": [42, 53, 62, 81, 88, 99, 102, 145, 154],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
            "temp", "wspd_lag_2_h", "started_at_month", "prcp", "temp_lag_2_h",
            "wspd_lag_1_h", "dwpt", "temp_lag_1_h", "wdir"
        ],

        "most_optimal": [ #8.278663649012222
          'started_at_quarter', 'prcp_lag_2_h', 'wspd', 'rhum', 'temp', 'prcp',
          'dwpt', 'temp_lag_1_h', 'wdir'
        ]
    },

    "2.25 - 3.0": {
        "clusters": [34, 39, 57, 59, 60, 72, 77, 89, 92, 95, 97, 106,
                     113, 118, 123, 163],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
            "temp", "started_at_month",  "wspd_lag_2_h", "wspd_lag_1_h" "prcp",
            "temp_lag_2_h", "temp_lag_1_h", "dwpt", "rhum_lag_24_h",
            "wdir"
        ],

        "most_optimal": [ #5.653652600413235
            'started_at_quarter', 'prcp_lag_2_h', 'is_holiday', 'rhum',
            'temp', 'started_at_month', 'wspd_lag_1_hprcp', 'temp_lag_1_h',
            'dwpt', 'wdir'
        ]
    },

    "1.5 - 2.25": {
        "clusters": [3, 19, 25, 43, 46, 51, 61, 64, 73, 75, 76, 82, 84, 90, 98,
                     108, 112, 114, 115, 116, 120, 129, 130, 136, 140, 141, 144,
                     148, 149, 152, 155, 160, 164],
        "features": [ #2.1155052503124523
            "started_at_quarter", "prcp_lag_2_h", "started_at_month", "wspd",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h", "temp_lag_1_h",
            "wspd_lag_1_h", 'dwpt', "temp_lag_2_h", "prcp"
        ],

        "most_optimal": [
            'prcp_lag_2_h', 'rhum', 'wspd_lag_2_h', 'wspd_lag_1_h', 'dwpt',
            'temp_lag_2_h'
        ]
    },

    "0 - 1.5": {
        "clusters": [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                     18, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 36,
                     37, 38, 40, 41, 44, 45, 47, 48, 50, 52, 54, 55, 56, 58, 63,
                     66, 67, 68, 70, 74, 78, 80, 83, 85, 87, 93, 96, 100, 101,
                     103, 104, 107, 109, 110, 111, 117, 121, 124, 125, 126, 127,
                     128, 131, 132, 133, 134, 135, 137, 138, 139, 142, 143, 146,
                     147, 150, 151, 153, 156, 157, 158, 159, 161, 162],
        "features": [
            "started_at_quarter", "wspd", "started_at_month", "prcp_lag_2_h",
            "rhum", "is_holiday", "wspd_lag_2_h", "temp", "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "prcp", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'rhum', 'dwpt', 'temp_lag_1_h', 'wspd_lag_1_h', 'cosine_sim_pca_0'
        ]
    }
}

#### 0.5


In [46]:
exclude_features_0_5 = {

    "4.5 - inf": {
        "clusters": [35, 65, 79, 94],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "prcp",
            "prcp_lag_1_h", "wspd_lag_1_h", "wspd", "wspd_lag_24_h",
            "temp_lag_2_h", "wspd_lag_2_h", "started_at_month",
            "cosine_sim_pca_0"
        ],

        "most_optimal": [
          'started_at_quarter', 'is_holiday', 'prcp', 'wspd', 'wspd_lag_24_h',
          'temp_lag_2_h'
        ]
    },

    "4.0 - 4.5": {
        "clusters": [69, 71, 86, 91],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "prcp",
            "wspd_lag_2_h", "temp_lag_2_h", "rhum", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'started_at_quarter', 'prcp_lag_2_h', 'wspd', 'prcp',
            'cosine_sim_pca_0'
        ]
    },

    "3.5 - 4.0": {
        "clusters": [105, 122, 154],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
            "wspd_lag_2_h", "prcp", "temp", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'started_at_quarter', 'prcp_lag_2_h', 'wspd', 'rhum',
             'wspd_lag_2_h', 'prcp', 'temp', 'cosine_sim_pca_0'
        ]
    },

    "3.0 - 3.5": {
        "clusters": [42, 53, 62, 81, 88, 99, 102, 145],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "is_holiday", "wspd", "rhum",
            "temp", "started_at_month", "wspd_lag_2_h", "prcp", "temp_lag_2_h",
            "wspd_lag_1_h", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'prcp_lag_2_h', 'wspd', 'started_at_month', 'wspd_lag_2_h', 'prcp',
            'wspd_lag_1_h', 'cosine_sim_pca_0'
        ]
    },

    "2.5 - 3.0": {
        "clusters": [34, 39, 57, 59, 60, 72, 77, 89, 97, 113, 118, 163],
        "features": [
            'started_at_quarter', 'prcp_lag_2_h', 'rhum',
            "cosine_sim_pca_0", "started_at_month",
            "is_holiday", "wspd",
            "temp",
        ],

        "most_optimal": [
            'prcp_lag_2_h', 'cosine_sim_pca_0', 'started_at_month'
        ]
    },

    "2.0 - 2.5": {
        "clusters": [64, 84, 92, 95, 98, 106, 123, 130, 144, 155],
        "features": [
            "started_at_quarter", "prcp_lag_2_h", "wspd", "is_holiday", "rhum",
            "started_at_month", "temp", "wspd_lag_2_h", "temp_lag_1_h",
            "wspd_lag_1_h", "prcp", "temp_lag_2_h", "dwpt", "rhum_lag_24_h",
            "cosine_sim_pca_0"
        ],
        "most_optimal": [
            'started_at_month', 'temp', 'wspd_lag_1_h'
        ],
    },

    "1.5 - 2.0": {
        "clusters": [3, 19, 25, 43, 46, 51, 61, 73, 75, 76, 82, 90, 108, 112,
                     114, 115, 116, 120, 129, 136, 140, 141, 148, 149, 152, 160,
                     164],
        "features": [
            "started_at_quarter", "started_at_month", "prcp_lag_2_h", "wspd",
            "is_holiday", "rhum", "temp", "wspd_lag_2_h", "temp_lag_1_h",
            "wspd_lag_1_h", "dwpt", "temp_lag_2_h", "prcp", "cosine_sim_pca_0"
        ],
        "most_optimal": [
            'rhum', 'wspd_lag_2_h', 'dwpt', 'prcp', 'cosine_sim_pca_0'
        ],
    },

    "0 - 1.5": {
        "clusters": [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                     18, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 36,
                     37, 38, 40, 41, 44, 45, 47, 48, 50, 52, 54, 55, 56, 58, 63,
                     66, 67, 68, 70, 74, 78, 80, 83, 85, 87, 93, 96, 100, 101,
                     103, 104, 107, 109, 110, 111, 117, 121, 124, 125, 126, 127,
                     128, 131, 132, 133, 134, 135, 137, 138, 139, 142, 143, 146,
                     147, 150, 151, 153, 156, 157, 158, 159, 161, 162],
        "features": [
            "started_at_quarter", "wspd", "started_at_month", "prcp_lag_2_h",
            "rhum", "is_holiday", "wspd_lag_2_h", "temp", "dwpt",
            "temp_lag_1_h", "wspd_lag_1_h", "prcp", "cosine_sim_pca_0"
        ],

        "most_optimal": [
            'rhum', 'dwpt', 'temp_lag_1_h', 'wspd_lag_1_h', 'cosine_sim_pca_0'
        ]
    }
}

## XGBoost

In [47]:
def print_results(overall_out, non_zero_out, zero_out):
  print("overall")
  print("MSE:", overall_out['mse'])
  print("RMSE:", overall_out['rmse'])
  print("MAE:", overall_out['mae'])
  print("MAPE:", overall_out['mape'])

  print()
  print("Non-zero")
  print("MSE:", non_zero_out['mse'])
  print("RMSE:", non_zero_out['rmse'])
  print("MAE:", non_zero_out['mae'])
  print("MAPE:", non_zero_out['mape'])

  print()
  print("Zeros")
  print("MSE:", zero_out['mse'])
  print("RMSE:", zero_out['rmse'])
  print("MAE:", zero_out['mae'])
  print("MAPE:", zero_out['mape'])
  print("###############")

### Regular Model

In [48]:
model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
    df_train, df_test, FEATURES, model='xgb'
)
print_results(overall_out, non_zero_out, zero_out)

overall
MSE: 3.442362569833841
RMSE: 1.855360495923593
MAE: 0.9865200162158131
MAPE: 0.39381197224261316

Non-zero
MSE: 7.025476542131613
RMSE: 2.6505615522246626
MAE: 1.6755732928438762
MAPE: 0.35334601894478496

Zeros
MSE: 0.5312117917119322
RMSE: 0.7288427757149907
MAE: 0.426689098770317
MAPE: 0.426689098770317
###############


In [49]:
preds_holdout = test_given_model(df_hold_out, model, FEATURES)

overall
MSE: 3.960625456129
RMSE: 1.990132019773814
MAE: 1.0650003488289315
MAPE: 0.4079845522311964

Non-zero
MSE: 7.800852567675758
RMSE: 2.7930006386815878
MAE: 1.7542552801176494
MAPE: 0.3521580775016717

#####
Zeros
MSE: 0.5742976610325577
RMSE: 0.7578242943008344
MAE: 0.45721256338460303
MAPE: 0.45721256338460303


In [50]:
def divide_excludes_test(df_train, df_test, clusters, df_hold_out=None):
  train = df_train[df_train['start_station_cluster'].isin(clusters)]
  train = pd.concat([train, df_train[~df_train['start_station_cluster'].isin(all_test_clusters)]])
  test = df_test[df_test['start_station_cluster'].isin(clusters)]
  if df_hold_out is not None:
    hold_out = df_hold_out[df_hold_out['start_station_cluster'].isin(clusters)]
    return train, test, hold_out
  return train, test

### Default Test

In [51]:
exclude_features = exclude_features_slope
all_test_clusters = []
for i in exclude_features:
  all_test_clusters.extend(exclude_features[i]['clusters'])
print(len(all_test_clusters))

163


In [52]:
excludes = '3.85 - inf'
#for excludes in exclude_features:
clusters = exclude_features[excludes]['clusters']
train, test = divide_excludes_test(df_train, df_test, clusters)
print(f"{excludes}")
model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
    train, test, FEATURES, model='xgb'
)
print_results(overall_out, non_zero_out, zero_out)
print("###############")

3.85 - inf
overall
MSE: 22.732583657718017
RMSE: 4.767869928775115
MAE: 3.281457560988409
MAPE: 0.41345364649130184

Non-zero
MSE: 24.031633222315758
RMSE: 4.902206974650882
MAE: 3.407041969023821
MAPE: 0.341219510156972

Zeros
MSE: 3.8987617414651874
RMSE: 1.9745282326330984
MAE: 1.4607153500248145
MAPE: 1.4607153500248145
###############
###############


In [53]:
del model
del model_preds
del preds_list
del train
del test

In [54]:
#MSE: 2.6457485592667016.
# 2.6397826606287103
#removed: Month, "temp_lag_2_h"#, "wspd_lag_2_h"


In [55]:
from itertools import combinations

def generate_combinations(lst):
    for r in range(len(lst), 0, -1):  # Start by removing the most elements
        for combo in combinations(lst, r):
            new_list = [x for x in lst if x not in combo]
            yield new_list

import random
def generate_combinations(lst):
  all_combos = []

  # Generate all possible combinations with at least one missing element
  for r in range(len(lst) - 1, 0, -1):  # Ensure at least one element is missing
      for combo in combinations(lst, r):
          new_list = [x for x in lst if x not in combo]
          all_combos.append(new_list)
  random.shuffle(all_combos)

  # Yield results
  for combo in all_combos:
      yield combo

### Exclude All

In [56]:
excludes = '3.85 - inf'
#for excludes in exclude_features:
clusters = exclude_features[excludes]['clusters']
excluded_feats = exclude_features[excludes]['features']
train, test = divide_excludes_test(df_train, df_test, clusters)

best = 2000000000000
best_comb = []
m = len(excluded_feats)
features = [i for i in FEATURES if i not in excluded_feats]
print(excluded_feats)
print(f"{excludes}")
model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
    train, test, features, model='xgb'
)
print_results(overall_out, non_zero_out, zero_out)

del model
del model_preds
del preds_list
del train
del test

['started_at_quarter', 'prcp_lag_2_h', 'is_holiday', 'prcp', 'wspd', 'wspd_lag_1_h', 'wspd_lag_2_h', 'temp_lag_2_h', 'started_at_month', 'rhum', 'dwpt', 'wspd_lag_1_h', 'temp', 'temp_lag_24_h', 'cosine_sim_pca_0']
3.85 - inf
overall
MSE: 22.867903869634585
RMSE: 4.782039718533775
MAE: 3.278233428643573
MAPE: 0.4164191466330387

Non-zero
MSE: 24.16466061649826
RMSE: 4.915756362605683
MAE: 3.402714820699584
MAPE: 0.34350891984841453

Zeros
MSE: 4.067323580234807
RMSE: 2.0167606650851773
MAE: 1.4734829142964634
MAPE: 1.4734829142964634
###############


### Run Most Optimal

In [57]:
del overall_out, non_zero_out, zero_out

In [58]:
excludes = '3.85 - inf'
#for excludes in exclude_features:
clusters = exclude_features[excludes]['clusters']
excluded_feats = exclude_features[excludes]['most_optimal']
train, test = divide_excludes_test(df_train, df_test, clusters)

print(len(clusters))
print(train.shape)
print(test.shape)

#excluded_feats = ['started_at_quarter', 'prcp_lag_2_h', 'wspd', 'started_at_month', 'rhum', 'temp', 'wspd_lag_2_h', 'temp_lag_1_h', 'wspd_lag_1_h', 'temp_lag_2_h']
features = [i for i in FEATURES if i not in excluded_feats]
print(excluded_feats)
print(f"{excludes}")
model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
    train, test, features, model='xgb'
)
print_results(overall_out, non_zero_out, zero_out)

del model
del model_preds
del preds_list
del train
del test

10
(76320, 148)
(16800, 148)
['started_at_quarter', 'prcp_lag_2_h', 'prcp', 'wspd_lag_2_h', 'temp', 'cosine_sim_pca_0']
3.85 - inf
overall
MSE: 21.627476640892645
RMSE: 4.650535091889174
MAE: 3.222624304166862
MAPE: 0.41783045448530615

Non-zero
MSE: 22.83351432833661
RMSE: 4.778442667683333
MAE: 3.339398925178195
MAPE: 0.34114640070312874

Zeros
MSE: 4.142155334740022
RMSE: 2.0352285706377113
MAE: 1.5296077508328145
MAPE: 1.5296077508328145
###############


### Test All

In [59]:
def get_rmses(X):


  print("Overall")
  print("MSE: ", (mean_squared_error(X['actual_demand'], X['pred'])))
  print("RMSE: ", np.sqrt(mean_squared_error(X['actual_demand'], X['pred'])))
  print("MAE: ", (mean_absolute_error(X['actual_demand'], X['pred'])))
  print("MAPE: ", (mean_absolute_percentage_error(X['actual_demand']+1, X['pred']+1)))
  print()

  print("Non-Zero")
  non_zeros = X[(X['actual_demand'] != 0)]
  print("MSE:", (mean_squared_error(non_zeros['actual_demand'], non_zeros['pred'])))
  print("RMSE:", np.sqrt(mean_squared_error(non_zeros['actual_demand'], non_zeros['pred'])))
  print("MAE:", (mean_absolute_error(non_zeros['actual_demand'], non_zeros['pred'])))
  print("MAPE:", (mean_absolute_percentage_error(non_zeros['actual_demand']+1, non_zeros['pred']+1)))
  print()

  print("Zeroes")
  zeros = X[(X['actual_demand'] == 0)]
  print("MSE:", (mean_squared_error(zeros['actual_demand'], zeros['pred'])))
  print("RMSE:", np.sqrt(mean_squared_error(zeros['actual_demand'], zeros['pred'])))
  print("MAE:", (mean_absolute_error(zeros['actual_demand'], zeros['pred'])))
  print("MAPE:", (mean_absolute_percentage_error(zeros['actual_demand']+1, zeros['pred']+1)))
  print()

  print("Under-predicting")
  under_predicting = X[(X['actual_demand'] > X['pred'])]
  print("MSE:", (mean_squared_error(under_predicting['actual_demand'], under_predicting['pred'])))
  print("RMSE:", np.sqrt(mean_squared_error(under_predicting['actual_demand'], under_predicting['pred'])))
  print("MAE:", (mean_absolute_error(under_predicting['actual_demand'], under_predicting['pred'])))
  print("MAPE:", (mean_absolute_percentage_error(under_predicting['actual_demand']+1, under_predicting['pred']+1)))
  print()


  print("Bver-predicting")
  over_predicting = X[(X['actual_demand'] < X['pred'])]
  print("MSE:", (mean_squared_error(over_predicting['actual_demand'], over_predicting['pred'])))
  print("RMSE:", np.sqrt(mean_squared_error(over_predicting['actual_demand'], over_predicting['pred'])))
  print("MAE:", (mean_absolute_error(over_predicting['actual_demand'], over_predicting['pred'])))
  print("MAPE:", (mean_absolute_percentage_error(over_predicting['actual_demand']+1, over_predicting['pred']+1)))

In [60]:
def test_all_excludes(exclude_features, mode='opt'):
  all_tests = pd.DataFrame()
  all_hold_out = pd.DataFrame()
  for excludes in exclude_features:
    clusters = exclude_features[excludes]['clusters']
    if mode == 'opt':
      excluded_feats = exclude_features[excludes]['most_optimal']
    if mode == 'default':
      excluded_feats = []
    if mode == 'all':
      excluded_feats = exclude_features[excludes]['features']

    train, test, hold_out = divide_excludes_test(df_train, df_test, clusters, df_hold_out)
    print(gc.collect())

    features = [i for i in FEATURES if i not in excluded_feats]
    if 'model' in exclude_features[excludes]:
      used_model = exclude_features[excludes]['model']
    else:
      used_model = 'xgb'
    model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
        train, test, features, model = used_model
    )
    preds_holdout = test_given_model(hold_out, model, features)
    print("*****************")
    print("*****************")
    print("*****************")
    all_hold_out = pd.concat([all_hold_out, preds_holdout])
    all_tests = pd.concat([all_tests, model_preds])
    #print_results(overall_out, non_zero_out, zero_out)

  del model
  del model_preds
  del preds_list
  del train
  del test

  return all_tests, all_hold_out

In [61]:
# overall
# MSE: 3.4385015247567643
# RMSE: 1.8543196932451438
# MAE: 0.9873231511704066
# MAPE: 0.39420786621054166

# Non-zero
# MSE: 7.019512252510148
# RMSE: 2.6494362140859606
# MAE: 1.6766834537912685
# MAPE: 0.3535476969635359

# Zeros
# MSE: 0.5290595569153295
# RMSE: 0.7273648031870455
# MAE: 0.4272427861871682
# MAPE: 0.4272427861871682

#### Slope Based

In [83]:
all_tests

,start_station_cluster,started_at_day,started_at_hour,started_at_week,started_at_dayofweek,temp,dwpt,rhum,wdir,pres,coco,demand_lag_1_h,demand_lag_2_h,demand_lag_24_h,temp_lag_1_h,temp_lag_24_h,prcp_lag_24_h,rhum_lag_1_h,rhum_lag_2_h,rhum_lag_24_h,mean_gnn_cluster_demand_1h,total_gnn_cluster_demand_1h,total_demand_1h,demand_degrees_1h,returns,actual_demand,pred,started_at_month,prcp_lag_1_h,wspd_lag_1_h,wspd_lag_24_h,temp_lag_2_h,prcp,wspd_lag_2_h
52280,35,1.000000,1.000000,1.000000,1.0,1.338983,1.644599,1.764706,1.055556,1.704545,1.176471,1.022535,1.011268,1.002817,1.423780,1.457317,1.0,1.74,1.77,1.75,1.054772,1.047619,1.195423,1.560,0.652176,12.0,2.874200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52281,35,1.000000,1.043478,1.000000,1.0,1.359322,1.623693,1.694118,1.027778,1.697727,1.176471,1.033803,1.022535,1.011268,1.405488,1.423780,1.0,1.80,1.74,1.77,1.120088,1.118326,1.287852,1.544,0.897161,6.0,7.885566,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52282,35,1.000000,1.260870,1.000000,1.0,1.284746,1.609756,1.800000,1.000000,1.725000,1.117647,1.000000,1.005634,1.000000,1.338415,1.390244,1.0,1.83,1.80,1.80,1.024896,1.001443,1.019366,1.136,0.947548,0.0,0.567756,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52283,35,1.000000,1.304348,1.000000,1.0,1.264407,1.606272,1.835294,1.000000,1.731818,1.117647,1.000000,1.000000,1.005634,1.356707,1.356707,1.0,1.83,1.83,1.83,1.002455,1.020202,1.017606,1.136,0.998246,0.0,0.943213,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52284,35,1.000000,1.347826,1.000000,1.0,1.247458,1.592334,1.835294,1.555556,1.736364,1.117647,1.000000,1.000000,1.005634,1.338415,1.356707,1.0,1.86,1.83,1.83,1.021340,1.008658,1.031690,1.224,1.107282,1.0,2.082702,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278495,162,1.766667,1.826087,1.215686,2.0,NaN,NaN,NaN,1.833333,1.579545,1.117647,1.002817,1.002817,1.000000,NaN,1.439024,1.0,1.65,1.75,1.62,1.014774,1.077922,1.368838,1.712,1.072971,0.0,0.419511,NaN,1.0,NaN,1.347990,1.371951,NaN,NaN
278496,162,1.766667,1.869565,1.215686,2.0,NaN,NaN,NaN,1.805556,1.572727,1.117647,1.000000,1.002817,1.000000,NaN,1.457317,1.0,1.65,1.65,1.60,1.009575,1.050505,1.231514,1.624,0.377685,0.0,0.216675,NaN,1.0,NaN,1.302764,1.439024,NaN,NaN
278497,162,1.766667,1.913043,1.215686,2.0,NaN,NaN,NaN,1.777778,1.561364,1.117647,1.000000,1.000000,1.002817,NaN,1.457317,1.0,1.62,1.65,1.58,1.177830,1.072150,1.161972,1.552,0.141066,0.0,0.157766,NaN,1.0,NaN,1.280151,1.439024,NaN,NaN
278498,162,1.766667,1.956522,1.215686,2.0,NaN,NaN,NaN,1.833333,1.561364,1.117647,1.000000,1.000000,1.000000,NaN,1.457317,1.0,1.64,1.62,1.58,1.054772,1.047619,1.115317,1.440,0.166026,0.0,0.115659,NaN,1.0,NaN,1.280151,1.475610,NaN,NaN


In [62]:
all_tests, all_hold_out = test_all_excludes(exclude_features_slope, "opt")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

488
overall
MSE: 25.179340151401192
RMSE: 5.017901967097523
MAE: 3.476899143244539
MAPE: 0.37991874255119545

Non-zero
MSE: 26.326394914429553
RMSE: 5.130925346799499
MAE: 3.5833589920058992
MAPE: 0.3258288936056881

#####
Zeros
MSE: 3.0528200124049527
RMSE: 1.7472321003246685
MAE: 1.423304089601489
MAPE: 1.423304089601489
*****************
*****************
*****************
503
overall
MSE: 8.446177595874794
RMSE: 2.90623082288293
MAE: 2.0052401933795916
MAPE: 0.526525825603909

Non-zero
MSE: 9.945300972344437
RMSE: 3.153617125198371
MAE: 2.2278443641337815
MAPE: 0.3851739832368397

#####
Zeros
MSE: 2.355399600910081
RMSE: 1.5347311168116977
MAE: 1.1008232480512474
MAPE: 1.1008232480512474
*****************
*****************
*****************
27
overall
MSE: 1.1712270455584333
RMSE: 1.082232436012908
MAE: 0.6521192590221936
MAPE: 0.3644862482686609

Non-zero
MSE: 2.5101685133468337
RMSE: 1.5843511332235773
MAE: 1.1110199143465516
MAPE: 0.3264415540650983

#####
Zeros
MSE: 0.396244915

In [63]:
all = pd.concat([all_tests, all_hold_out])
get_rmses(all)

Overall
MSE:  3.270936718019814
RMSE:  1.8085731165810837
MAE:  0.9597029196140414
MAPE:  0.377083542569962

Non-Zero
MSE: 6.688071370785735
RMSE: 2.586130578835055
MAE: 1.6383998274238936
MAPE: 0.34321087687299384

Zeroes
MSE: 0.47697955693544275
RMSE: 0.6906370660017045
MAE: 0.40477890330514943
MAPE: 0.40477890330514943

Under-predicting
MSE: 6.663153242366959
RMSE: 2.581308436116645
MAE: 1.5367024822398738
MAPE: 0.27653122024682764

Bver-predicting
MSE: 1.6200719343579166
RMSE: 1.2728204643066974
MAE: 0.6789052070868843
MAPE: 0.42602608328033525


In [64]:
all_tests, all_hold_out = test_all_excludes(exclude_features_slope, "default")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 27.09862569350462
RMSE: 5.2056340337661675
MAE: 3.5872120504294123
MAPE: 0.38420211182855213

Non-zero
MSE: 28.337844672049215
RMSE: 5.323330223840074
MAE: 3.7004587090525924
MAPE: 0.33140242502464035

#####
Zeros
MSE: 3.1942711943328597
RMSE: 1.78725241483481
MAE: 1.402700418147488
MAPE: 1.402700418147488
*****************
*****************
*****************
27
overall
MSE: 8.587033304125828
RMSE: 2.9303640224596377
MAE: 2.0087934500312215
MAPE: 0.5151788655934029

Non-zero
MSE: 10.150795807271303
RMSE: 3.186031356919028
MAE: 2.2504873907326175
MAPE: 0.38924940238518135

#####
Zeros
MSE: 2.233633448578803
RMSE: 1.4945345257232445
MAE: 1.0268168104519635
MAPE: 1.0268168104519635
*****************
*****************
*****************
1149
overall
MSE: 1.189245082278074
RMSE: 1.090525140598819
MAE: 0.6524574184855381
MAPE: 0.3630371816933907

Non-zero
MSE: 2.554905129888389
RMSE: 1.5984070601346796
MAE: 1.1157424874098043
MAPE: 0.32628909949798585

#####
Zeros
MSE: 0.398798

In [65]:
all = pd.concat([all_tests, all_hold_out])
get_rmses(all)

Overall
MSE:  3.367956425303152
RMSE:  1.8351992876260474
MAE:  0.9646209745492749
MAPE:  0.37506101846940243

Non-Zero
MSE: 6.910103336427866
RMSE: 2.6287075410604097
MAE: 1.6551552395268259
MAPE: 0.34453707115074494

Zeroes
MSE: 0.4717852955119392
RMSE: 0.6868662864866343
MAE: 0.40001836134920243
MAPE: 0.40001836134920243

Under-predicting
MSE: 7.035757644085344
RMSE: 2.6525002627870453
MAE: 1.5677703244696233
MAPE: 0.2789877342707121

Bver-predicting
MSE: 1.567593556040277
RMSE: 1.2520357646809763
MAE: 0.6685830896587592
MAPE: 0.4222436527875138


In [66]:
all_tests, all_hold_out = test_all_excludes(exclude_features_slope, "all")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 27.05473629823402
RMSE: 5.201416758752755
MAE: 3.548972923500197
MAPE: 0.3839568930571256

Non-zero
MSE: 28.302299110554145
RMSE: 5.319990517900774
MAE: 3.660450058803802
MAPE: 0.3313573145361082

#####
Zeros
MSE: 2.989430454783495
RMSE: 1.7289969504841514
MAE: 1.3985951396002285
MAPE: 1.3985951396002285
*****************
*****************
*****************
27
overall
MSE: 8.535413599943709
RMSE: 2.92154301695931
MAE: 1.9968011803693653
MAPE: 0.5136422884252071

Non-zero
MSE: 10.076037256841687
RMSE: 3.174277438542776
MAE: 2.2318514239954137
MAPE: 0.3836425880588078

#####
Zeros
MSE: 2.2760244027355725
RMSE: 1.508649860880772
MAE: 1.0418171716748543
MAPE: 1.0418171716748543
*****************
*****************
*****************
745
overall
MSE: 1.179080741448022
RMSE: 1.0858548436361197
MAE: 0.6537478867180142
MAPE: 0.36458171545220075

Non-zero
MSE: 2.528262080877421
RMSE: 1.5900509680124788
MAE: 1.1164825226767532
MAPE: 0.32772215088826817

#####
Zeros
MSE: 0.3981717519

#### 2.0

In [67]:
all_tests, all_hold_out = test_all_excludes(exclude_features_2, "opt")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 29.265092529486406
RMSE: 5.409722038098298
MAE: 3.674782850007926
MAPE: 0.3857071032238634

Non-zero
MSE: 30.421803202238063
RMSE: 5.51559635961861
MAE: 3.774187581712792
MAPE: 0.3346312452065832

#####
Zeros
MSE: 3.9827021107717946
RMSE: 1.995670842291332
MAE: 1.5020794284587005
MAPE: 1.5020794284587005
*****************
*****************
*****************
510
overall
MSE: 7.784433098864914
RMSE: 2.7900596944984732
MAE: 1.8842583990200257
MAPE: 0.5075755467837539

Non-zero
MSE: 9.43383014122468
RMSE: 3.0714540760403173
MAE: 2.1398857236734745
MAPE: 0.3765713967171184

#####
Zeros
MSE: 1.9114073024269371
RMSE: 1.3825365465067956
MAE: 0.9740434335461962
MAPE: 0.9740434335461962
*****************
*****************
*****************
27
overall
MSE: 1.0469176367132431
RMSE: 1.0231899318861788
MAE: 0.6149631598309389
MAPE: 0.3590265993378262

Non-zero
MSE: 2.3007956969493644
RMSE: 1.516837399640899
MAE: 1.067273154639128
MAPE: 0.3246497048725713

#####
Zeros
MSE: 0.3875342732

In [68]:
all = pd.concat([all_tests, all_hold_out])
get_rmses(all)

Overall
MSE:  3.3142053618415397
RMSE:  1.8204959109653445
MAE:  0.9605329648525989
MAPE:  0.3757311811677276

Non-Zero
MSE: 6.787714259493583
RMSE: 2.60532421389231
MAE: 1.6441587726531108
MAPE: 0.34411823446517864

Zeroes
MSE: 0.4741548385676474
RMSE: 0.6885890200748538
MAE: 0.40157892432033426
MAPE: 0.40157892432033426

Under-predicting
MSE: 6.881233137791224
RMSE: 2.6232104638765117
MAE: 1.556944738888493
MAPE: 0.2789334279714728

Bver-predicting
MSE: 1.604831177399543
RMSE: 1.2668193152141085
MAE: 0.6747297235047395
MAPE: 0.4221254635456983


In [69]:
all_tests, all_hold_out = test_all_excludes(exclude_features_2, "default")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

430
overall
MSE: 28.293157690342458
RMSE: 5.31913129094803
MAE: 3.6499099157750607
MAPE: 0.37994130841963564

Non-zero
MSE: 29.422976338997305
RMSE: 5.424295008477812
MAE: 3.747379123218476
MAPE: 0.3278041089905804

#####
Zeros
MSE: 3.5985500840292635
RMSE: 1.896984471214581
MAE: 1.5195115245118433
MAPE: 1.5195115245118433
*****************
*****************
*****************
27
overall
MSE: 7.88616991004514
RMSE: 2.808232524212541
MAE: 1.8876082842097137
MAPE: 0.4983224107115764

Non-zero
MSE: 9.589009235475652
RMSE: 3.0966125420329313
MAE: 2.1563849308036653
MAPE: 0.3769281147345236

#####
Zeros
MSE: 1.8228516012318563
RMSE: 1.3501302163983504
MAE: 0.9305723866140736
MAPE: 0.9305723866140736
*****************
*****************
*****************
1149
overall
MSE: 1.0300273212942732
RMSE: 1.0149026166555455
MAE: 0.6074882212571076
MAPE: 0.35136911313664604

Non-zero
MSE: 2.2963328763904642
RMSE: 1.5153655916611226
MAE: 1.064604421460249
MAPE: 0.3214512929775786

#####
Zeros
MSE: 0.3641

In [70]:
all_tests, all_hold_out = test_all_excludes(exclude_features_2, "all")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

561
overall
MSE: 28.310881513086297
RMSE: 5.320797074977235
MAE: 3.6232156091502734
MAPE: 0.3821848020055255

Non-zero
MSE: 29.43155212176107
RMSE: 5.425085448337295
MAE: 3.7169520892794172
MAPE: 0.32763882690582447

#####
Zeros
MSE: 3.8162239234806177
RMSE: 1.9535157853164682
MAE: 1.5744039720418501
MAPE: 1.5744039720418501
*****************
*****************
*****************
27
overall
MSE: 7.920030258714921
RMSE: 2.8142548318720038
MAE: 1.89859571401433
MAPE: 0.5047216491981791

Non-zero
MSE: 9.60184587216606
RMSE: 3.0986845389884494
MAE: 2.163316646390983
MAPE: 0.37798307848119195

#####
Zeros
MSE: 1.9315713073642342
RMSE: 1.3898098097812643
MAE: 0.9560010416721898
MAPE: 0.9560010416721898
*****************
*****************
*****************
758
overall
MSE: 1.039221417195467
RMSE: 1.0194220996208916
MAE: 0.6111784341106391
MAPE: 0.3546560778435765

Non-zero
MSE: 2.30062034726534
RMSE: 1.5167795974581606
MAE: 1.0676584857719507
MAPE: 0.32333529573510483

#####
Zeros
MSE: 0.375883

#### 1.5

In [71]:
all_tests, all_hold_out = test_all_excludes(exclude_features_1_5, "opt")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 40.739547735448845
RMSE: 6.382753930353954
MAE: 4.471296810890947
MAPE: 0.35958755217407207

Non-zero
MSE: 41.58406678295504
RMSE: 6.448570910128463
MAE: 4.5432002768464335
MAPE: 0.341453589106561

#####
Zeros
MSE: 2.173177899332638
RMSE: 1.4741702409601944
MAE: 1.1877051989237468
MAPE: 1.1877051989237468
*****************
*****************
*****************
503
overall
MSE: 13.398559111131705
RMSE: 3.660404227832181
MAE: 2.5268668508245833
MAPE: 0.4652629593363715

Non-zero
MSE: 15.203431577728304
RMSE: 3.899157803645334
MAE: 2.772853630685304
MAPE: 0.3716510948421288

#####
Zeros
MSE: 2.4417070664392235
RMSE: 1.5625962582955406
MAE: 1.0335531670236426
MAPE: 1.0335531670236426
*****************
*****************
*****************
500
overall
MSE: 3.895946230436391
RMSE: 1.9738151459638744
MAE: 1.3173979712123427
MAPE: 0.5247599611115992

Non-zero
MSE: 5.195637681819098
RMSE: 2.279394147974215
MAE: 1.5672587342220952
MAPE: 0.35919458196901816

#####
Zeros
MSE: 1.41611951

In [72]:
all = pd.concat([all_tests, all_hold_out])
get_rmses(all)

Overall
MSE:  3.3221357542401755
RMSE:  1.82267269531317
MAE:  0.9608571731980451
MAPE:  0.37668187122089286

Non-Zero
MSE: 6.789482676071068
RMSE: 2.6056635769168413
MAE: 1.6451123171098856
MAPE: 0.34646447579667217

Zeroes
MSE: 0.48712345691327325
RMSE: 0.6979423019944222
MAE: 0.4013885675948515
MAPE: 0.4013885675948515

Under-predicting
MSE: 6.7159103760733965
RMSE: 2.5915073559751662
MAE: 1.525359792138971
MAPE: 0.27327343456394776

Bver-predicting
MSE: 1.6780576867310386
RMSE: 1.2953986593829092
MAE: 0.6874049051755232
MAPE: 0.42679458320220404


In [73]:
all_tests, all_hold_out = test_all_excludes(exclude_features_1_5, "default")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 39.51321380113654
RMSE: 6.285953690661151
MAE: 4.337402159401349
MAPE: 0.35219024179388

Non-zero
MSE: 40.33998694309498
RMSE: 6.351376775400352
MAE: 4.409755694387603
MAPE: 0.3372763625259562

#####
Zeros
MSE: 1.7572403183678809
RMSE: 1.3256094139556647
MAE: 1.033257395029068
MAPE: 1.033257395029068
*****************
*****************
*****************
588
overall
MSE: 13.088660018197745
RMSE: 3.6178253161530263
MAE: 2.498248161361331
MAPE: 0.46122535940573206

Non-zero
MSE: 14.857180118684168
RMSE: 3.8545012801508016
MAE: 2.7425281308256646
MAPE: 0.3699558157359489

#####
Zeros
MSE: 2.3524925394872276
RMSE: 1.533783732958212
MAE: 1.015296023502093
MAPE: 1.015296023502093
*****************
*****************
*****************
588
overall
MSE: 3.886377070132796
RMSE: 1.9713896292039268
MAE: 1.3133977375642203
MAPE: 0.5201731773905112

Non-zero
MSE: 5.1983240006180855
RMSE: 2.2799833334079627
MAE: 1.5672465364844554
MAPE: 0.3582884198899997

#####
Zeros
MSE: 1.383166754696

In [74]:
all_tests, all_hold_out = test_all_excludes(exclude_features_1_5, "all")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 43.8762368474922
RMSE: 6.623914012688585
MAE: 4.4645360944526535
MAPE: 0.34982711643828523

Non-zero
MSE: 44.781684255747834
RMSE: 6.691911853554845
MAE: 4.534978381691188
MAPE: 0.3301662873699355

#####
Zeros
MSE: 2.5274718704849874
RMSE: 1.5898024627245322
MAE: 1.247671643892924
MAPE: 1.247671643892924
*****************
*****************
*****************
418
overall
MSE: 13.161306195150672
RMSE: 3.6278514571507294
MAE: 2.5039749259608133
MAPE: 0.4582728187867957

Non-zero
MSE: 14.951229151330079
RMSE: 3.8666819304579576
MAE: 2.750267476421955
MAPE: 0.36758615359032054

#####
Zeros
MSE: 2.295208249051437
RMSE: 1.5149944716240509
MAE: 1.0088049984138823
MAPE: 1.0088049984138823
*****************
*****************
*****************
425
overall
MSE: 3.830479823019211
RMSE: 1.9571611642936335
MAE: 1.3073982100966373
MAPE: 0.5183529190925292

Non-zero
MSE: 5.1193333077211625
RMSE: 2.2625943754286055
MAE: 1.5567579431497793
MAPE: 0.35416947474538446

#####
Zeros
MSE: 1.37133

#### 0.75

In [75]:

all_tests, all_hold_out = test_all_excludes(exclude_features_0_75, "opt")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 40.739547735448845
RMSE: 6.382753930353954
MAE: 4.471296810890947
MAPE: 0.35958755217407207

Non-zero
MSE: 41.58406678295504
RMSE: 6.448570910128463
MAE: 4.5432002768464335
MAPE: 0.341453589106561

#####
Zeros
MSE: 2.173177899332638
RMSE: 1.4741702409601944
MAE: 1.1877051989237468
MAPE: 1.1877051989237468
*****************
*****************
*****************
503
overall
MSE: 19.638535682877144
RMSE: 4.431538748885893
MAE: 3.2227923672823677
MAPE: 0.45048669126060503

Non-zero
MSE: 20.71936950831328
RMSE: 4.551853414633789
MAE: 3.33562526453196
MAPE: 0.3615042327844751

#####
Zeros
MSE: 4.7912920808333865
RMSE: 2.1889020263212755
MAE: 1.6728246734853376
MAPE: 1.6728246734853376
*****************
*****************
*****************
500
overall
MSE: 10.380406703563038
RMSE: 3.2218638555288206
MAE: 2.1459286268268314
MAPE: 0.4719290432371376

Non-zero
MSE: 12.350222057483649
RMSE: 3.5142882718245594
MAE: 2.440942820205408
MAPE: 0.37306098165343343

#####
Zeros
MSE: 2.0086914

In [76]:
all = pd.concat([all_tests, all_hold_out])
get_rmses(all)

Overall
MSE:  3.3705319826929796
RMSE:  1.8359008640700019
MAE:  0.9606348856449533
MAPE:  0.37072622451928433

Non-Zero
MSE: 6.925636516260863
RMSE: 2.6316604105128882
MAE: 1.6568925293289396
MAPE: 0.345499174029608

Zeroes
MSE: 0.4637662917578505
RMSE: 0.6810038852736822
MAE: 0.39135265734411234
MAPE: 0.39135265734411234

Under-predicting
MSE: 6.830023875318966
RMSE: 2.613431436888859
MAE: 1.5390482571995487
MAPE: 0.2756002447313766

Bver-predicting
MSE: 1.6599047932980162
RMSE: 1.288372924776835
MAE: 0.6746416001176434
MAPE: 0.41778313248863264


In [77]:
all_tests, all_hold_out = test_all_excludes(exclude_features_0_75, "default")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 39.51321380113654
RMSE: 6.285953690661151
MAE: 4.337402159401349
MAPE: 0.35219024179388

Non-zero
MSE: 40.33998694309498
RMSE: 6.351376775400352
MAE: 4.409755694387603
MAPE: 0.3372763625259562

#####
Zeros
MSE: 1.7572403183678809
RMSE: 1.3256094139556647
MAE: 1.033257395029068
MAPE: 1.033257395029068
*****************
*****************
*****************
588
overall
MSE: 19.971694472190634
RMSE: 4.468970180275388
MAE: 3.2520491089139667
MAPE: 0.4684170273766008

Non-zero
MSE: 21.061925901320702
RMSE: 4.589327390949648
MAE: 3.3568067491282907
MAPE: 0.3705347842606184

#####
Zeros
MSE: 4.995357472035478
RMSE: 2.2350296356056396
MAE: 1.8130099459698326
MAPE: 1.8130099459698326
*****************
*****************
*****************
588
overall
MSE: 10.240659330145686
RMSE: 3.200103018677006
MAE: 2.1429172744826666
MAPE: 0.475677655930262

Non-zero
MSE: 12.186212022384462
RMSE: 3.49087553808274
MAE: 2.4389166197940413
MAPE: 0.3793853262881301

#####
Zeros
MSE: 1.972060388130887

In [78]:

all_tests, all_hold_out = test_all_excludes(exclude_features_0_75, "all")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 40.856808757381465
RMSE: 6.391933100196017
MAE: 4.358669597336224
MAPE: 0.3497060451844823

Non-zero
MSE: 41.7103805231361
RMSE: 6.458357416800041
MAE: 4.428599701093061
MAPE: 0.3318486259015001

#####
Zeros
MSE: 1.8770314545867528
RMSE: 1.3700479752865418
MAE: 1.1651948591073353
MAPE: 1.1651948591073353
*****************
*****************
*****************
438
overall
MSE: 19.551245144421166
RMSE: 4.421678996085216
MAE: 3.1920299049644245
MAPE: 0.4434207378475845

Non-zero
MSE: 20.67168641059478
RMSE: 4.546612630365026
MAE: 3.306393914371919
MAPE: 0.35769442474465807

#####
Zeros
MSE: 4.159920382773089
RMSE: 2.0395882875651865
MAE: 1.6210295652088367
MAPE: 1.6210295652088367
*****************
*****************
*****************
380
overall
MSE: 10.407431743942052
RMSE: 3.2260551365316203
MAE: 2.160637103328629
MAPE: 0.4726879138104225

Non-zero
MSE: 12.407686052304385
RMSE: 3.5224545493596344
MAE: 2.465115563483799
MAPE: 0.3800018587848381

#####
Zeros
MSE: 1.9063509334

#### 0.5


In [79]:
all_tests, all_hold_out = test_all_excludes(exclude_features_0_5, "opt")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 40.739547735448845
RMSE: 6.382753930353954
MAE: 4.471296810890947
MAPE: 0.35958755217407207

Non-zero
MSE: 41.58406678295504
RMSE: 6.448570910128463
MAE: 4.5432002768464335
MAPE: 0.341453589106561

#####
Zeros
MSE: 2.173177899332638
RMSE: 1.4741702409601944
MAE: 1.1877051989237468
MAPE: 1.1877051989237468
*****************
*****************
*****************
503
overall
MSE: 17.220384972666086
RMSE: 4.14974516960573
MAE: 2.952454496600798
MAPE: 0.41035836378141943

Non-zero
MSE: 18.174227558622007
RMSE: 4.263124154727611
MAE: 3.049794672438109
MAPE: 0.3278561745817954

#####
Zeros
MSE: 3.737691122532321
RMSE: 1.9333109223640985
MAE: 1.5765379570625924
MAPE: 1.5765379570625924
*****************
*****************
*****************
493
overall
MSE: 16.789467922568598
RMSE: 4.097495323068545
MAE: 3.020232852300008
MAPE: 0.4670842794495393

Non-zero
MSE: 18.009667613023673
RMSE: 4.243779873299706
MAE: 3.1664943992317496
MAPE: 0.38124141066760187

#####
Zeros
MSE: 3.3672713275

In [80]:
all = pd.concat([all_tests, all_hold_out])
get_rmses(all)

Overall
MSE:  3.333445972478887
RMSE:  1.8257727055903994
MAE:  0.9572767863984197
MAPE:  0.37025191023544135

Non-Zero
MSE: 6.848001301899317
RMSE: 2.6168686061587647
MAE: 1.6495439835162482
MAPE: 0.3445614112540196

Zeroes
MSE: 0.45983459087492334
RMSE: 0.6781110461236591
MAE: 0.3912572731768388
MAPE: 0.3912572731768388

Under-predicting
MSE: 6.752892865982993
RMSE: 2.598632884034025
MAE: 1.5334600964271872
MAPE: 0.2758855046513226

Bver-predicting
MSE: 1.644044963374155
RMSE: 1.2822031677445487
MAE: 0.6726241578361755
MAPE: 0.4168914020228735


In [81]:
all_tests, all_hold_out = test_all_excludes(exclude_features_0_5, "default")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 39.51321380113654
RMSE: 6.285953690661151
MAE: 4.337402159401349
MAPE: 0.35219024179388

Non-zero
MSE: 40.33998694309498
RMSE: 6.351376775400352
MAE: 4.409755694387603
MAPE: 0.3372763625259562

#####
Zeros
MSE: 1.7572403183678809
RMSE: 1.3256094139556647
MAE: 1.033257395029068
MAPE: 1.033257395029068
*****************
*****************
*****************
27
overall
MSE: 18.474188065620346
RMSE: 4.2981610097366465
MAE: 3.0775727694588046
MAPE: 0.4319491804409717

Non-zero
MSE: 19.43043969072189
RMSE: 4.407997242594633
MAE: 3.1730600081483904
MAPE: 0.34026993195338734

#####
Zeros
MSE: 4.957442121617399
RMSE: 2.2265314104268548
MAE: 1.72784774690061
MAPE: 1.72784774690061
*****************
*****************
*****************
1149
overall
MSE: 17.8499692564132
RMSE: 4.224922396495964
MAE: 3.128340324333736
MAPE: 0.49280776868444576

Non-zero
MSE: 19.181409429384
RMSE: 4.379658597354822
MAE: 3.272646152199089
MAPE: 0.3975197278544087

#####
Zeros
MSE: 3.204127353734419
RMSE: 

In [82]:
all_tests, all_hold_out = test_all_excludes(exclude_features_0_5, "all")
get_rmses(all_tests)
print()
print()
get_rmses(all_hold_out)
print(gc.collect())

0
overall
MSE: 43.8762368474922
RMSE: 6.623914012688585
MAE: 4.4645360944526535
MAPE: 0.34982711643828523

Non-zero
MSE: 44.781684255747834
RMSE: 6.691911853554845
MAE: 4.534978381691188
MAPE: 0.3301662873699355

#####
Zeros
MSE: 2.5274718704849874
RMSE: 1.5898024627245322
MAE: 1.247671643892924
MAPE: 1.247671643892924
*****************
*****************
*****************
418
overall
MSE: 17.040562721187865
RMSE: 4.128021647373941
MAE: 2.9938998149974005
MAPE: 0.42085902922983165

Non-zero
MSE: 17.973315233202065
RMSE: 4.239494690785928
MAE: 3.0932721161705587
MAPE: 0.33819976429706233

#####
Zeros
MSE: 3.855979916230358
RMSE: 1.963664919539573
MAE: 1.5892589092254639
MAPE: 1.5892589092254639
*****************
*****************
*****************
457
overall
MSE: 17.489001984898334
RMSE: 4.181985411846666
MAE: 3.0363214015960693
MAPE: 0.4602560192163276

Non-zero
MSE: 18.797375045788556
RMSE: 4.335593966896411
MAE: 3.1829560143607005
MAPE: 0.3727028699464366

#####
Zeros
MSE: 3.09689831

### Optimize

In [ ]:

excludes = '3.85 - inf'
#for excludes in exclude_features:
clusters = exclude_features[excludes]['clusters']
excluded_feats = exclude_features[excludes]['features']
train, test = divide_excludes_test(df_train, df_test, clusters)



counter = 0
stop_counter = 0
best_mse = 20000000000000
best_comb = []

for combination in generate_combinations(excluded_feats):
  features = [i for i in FEATURES if i not in combination]

  model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
      train, test, features, model='xgb'
  )
  if overall_out['mse'] < best_mse:
    best_mse = overall_out['mse']
    best_comb = combination

    print("Best comb:", best_comb)
    print_results(overall_out, non_zero_out, zero_out)
    print(gc.collect())
    print("###############")
    stop_counter= 0
  else:
    stop_counter+=1
  del model
  del model_preds
  del preds_list
  del overall_out
  del non_zero_out
  del zero_out
  print(stop_counter)
  if stop_counter==1500:
    break
  if counter==5500:
    break
print(best_comb)
del train
del test

Best comb: ['prcp_lag_2_h', 'wspd_lag_2_h', 'started_at_month', 'temp']
overall
MSE: 22.331088190344115
RMSE: 4.725578080017736
MAE: 3.2597111925269875
MAPE: 0.41571268756138746

Non-zero
MSE: 23.585981971713927
RMSE: 4.856540123556473
MAE: 3.3821870084860515
MAPE: 0.3420257153184463

Zeros
MSE: 4.137443662661483
RMSE: 2.0340707123061095
MAE: 1.4840378312607092
MAPE: 1.4840378312607092
###############
464
###############
0
1
2
Best comb: ['started_at_quarter', 'is_holiday', 'wspd', 'wspd_lag_2_h', 'temp_lag_2_h', 'rhum', 'dwpt', 'cosine_sim_pca_0']
overall
MSE: 22.2172448031945
RMSE: 4.713517243332679
MAE: 3.2567535165661856
MAPE: 0.41670015051876647

Non-zero
MSE: 23.48173557103698
RMSE: 4.845795659232546
MAE: 3.3807468419199105
MAPE: 0.34480280084096926

Zeros
MSE: 3.8844616782752905
RMSE: 1.970903771947096
MAE: 1.4590790689101518
MAPE: 1.4590790689101518
###############
1869
###############
0
1
2
3
4
5
6
7
8
9
10
Best comb: ['started_at_quarter', 'prcp', 'wspd', 'wspd_lag_1_h', 'sta

In [ ]:

# excludes = '1.5 - 2.25'
# #for excludes in exclude_features:
# clusters = exclude_features[excludes]['clusters']
# excluded_feats = exclude_features[excludes]['features']
# train, test = divide_excludes_test(df_train, df_test, clusters)


# best = 2000000000000
# best_comb = []
# m = len(excluded_feats)
# flag = False
# current_size = 200000
# counters = 0
# size_counter = 0

# for combination in generate_combinations(excluded_feats):
#   features = [i for i in FEATURES if i not in combination]


#   if size_counter<=40:
#     size_counter += 1
#   elif size_counter>40:
#     size_counter += 1
#     if current_size!=len(combination):
#       size_counter = 0
#       print("*************************************")
#     else:
#       continue

#   #print(combination)
#   #print(features)
#   #print(f"{excludes}")
#   print(gc.collect())
#   print(counters, len(excluded_feats)*0.80)
#   if flag:
#     counters += 1
#   if flag and current_size==len(combination) and counters>len(excluded_feats)*0.80==0:
#     #counters += 1
#     continue
#   elif current_size!=len(combination):
#     flag = False
#     counters = 0
#     current_size = combination
#   model, model_preds, preds_list, overall_out, non_zero_out, zero_out = train_model(
#       train, test, features, model='xgb'
#   )
#   if overall_out['mse'] <= best:
#     best = overall_out['mse']
#     print("previous best: ", best_comb)
#     best_comb = combination
#     print(best_comb)
#     flag = True
#     current_size = len(combination)
#     counters = 0
#     size_counter = 0
#     print_results(overall_out, non_zero_out, zero_out)
#   print("###############")
#   del model
#   del model_preds
#   del preds_list
#   del overall_out
#   del non_zero_out
#   del zero_out
# print(best_comb)
# del train
# del test

Streaming output truncated to the last 5000 lines.
4 11.200000000000001
###############
519
5 11.200000000000001
###############
519
6 11.200000000000001
###############
25
7 11.200000000000001
###############
1013
8 11.200000000000001
###############
519
9 11.200000000000001
###############
526
10 11.200000000000001
###############
25
11 11.200000000000001
###############
1027
12 11.200000000000001
###############
526
13 11.200000000000001
###############
526
14 11.200000000000001
###############
526
15 11.200000000000001
###############
526
16 11.200000000000001
###############
525
17 11.200000000000001
###############
519
18 11.200000000000001
###############
519
19 11.200000000000001
###############
526
20 11.200000000000001
###############
526
21 11.200000000000001
###############
526
22 11.200000000000001
###############
526
23 11.200000000000001
###############
526
24 11.200000000000001
###############
526
25 11.200000000000001
###############
526
26 11.200000000000001
#########

In [ ]:
best_comb

In [ ]:
#aggregated_importance = []
#
#for i in range(df_test.shape[0]):
#    row = df_test[FEATURES].loc[i].values#.reshape(1, -1)
#    explanation = explainer.explain_instance(
#        data_row=row,
#        predict_fn=model.predict  # Prediction function of the model
#    )
#    # Extract feature importance
#    feature_importance = dict(explanation.as_list())
#    aggregated_importance.append(feature_importance)
#
## Aggregate feature importance
#importance_df = pd.DataFrame(aggregated_importance).fillna(0)  # Handle missing features
#mean_importance = importance_df.mean().sort_values(ascending=False)

In [ ]:
#11-15
3.519954687632939
1.8761542281041126
0.39561652953215365


#11-28 (20 dim)
3.524785445848623
1.877441196375701
0.39610593754659995

#11-29 (no dim)
3.563436345796625
1.887706636582238
0.4006454580255268

#11-29 (5 dim)
3.504096872035053
1.8719233082674762
0.39420467949842114

#11-29 (6 dim)
3.524516281760969
1.8773695112473114
0.39833392477077356

#11-29 (1 dim)
3.4851078325990033
1.8668443514655964
0.39848029011836106

0.39848029011836106

In [ ]:
suf = "_o_xgb_01_27"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## LightGBM

In [ ]:
#FEATURES.extend([f"dim_mean_{i}" for i in range(50)])
#FEATURES.extend([f"cosine_sim_pca_{i}" for i in range(PCA_DIM)])

model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='lgbm')

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.750807 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3954
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 37
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.169623 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3944
[LightGBM] [Info] Number of data points in the train set: 1140300, number of used features: 37
[LightGBM] [Info] Start training from score 1.005143
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.186575 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Tota

In [ ]:
#11-28
3.622206095204831
1.9032094196921239
0.3979671041602479

#11-29
3.5923026180976714
1.895337072422125
0.3977005786689495

0.3977005786689495

In [ ]:
suf = "_o_lgbm_01_27"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model

## Random Forest

In [ ]:
#FEATURES.extend([f"dim_mean_{i}" for i in range(50)])
#FEATURES.extend([f"cosine_sim_pca_{i}" for i in range(PCA_DIM)])

model, model_preds, preds_list = train_model(df_train, df_test, FEATURES, model='rf')

In [ ]:
#11_01
3.6042873699256828
1.8984960810930538
0.4084309933647605

#11_28
3.7858322010061154
1.9457215116778956
0.44009657232024235

In [ ]:
suf = "_o_rf_01_27"


model_preds.to_csv(f"{models_dir}/test_predictions{suf}.csv")
del model_preds

filename = f'{models_dir}/demand_model{suf}.sav'
joblib.dump(model, filename)
del model